In [67]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## LOAD MODEL

In [58]:
model = YOLO('./model/client_model/best.pt')
model2 = YOLO('./model/client_model/best.pt')

## READ FROM VIDEO

In [59]:
video_path = './dataset/Video/sample.mp4'
cap = cv2.VideoCapture(video_path)

## READ FROM WEBCAM

In [60]:
# cap = cv2.VideoCapture(0) 

## LICENSE DETECTION

In [61]:
captured = []
plate = 0

In [62]:
def create_detection_frame(frame):
    new_height = int(frame.shape[0] * 0.4)  # 40% dari tinggi frame asli
    new_width = frame.shape[1]  # Lebar sesuai dengan frame asli

    # Hitung koordinat untuk rectangle
    top_left_x = 0
    top_left_y = frame.shape[0] - new_height - int(frame.shape[0] * 0.1)  # Angkat rectangle ke tengah 10%
    bottom_right_x = frame.shape[1]
    bottom_right_y = top_left_y + new_height
    
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 255), thickness=2)
    
    return frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x], top_left_x, top_left_y, bottom_right_x, bottom_right_y

In [63]:
def detect_objects(frame, model):
    # Lakukan deteksi objek menggunakan model
    result = model.track(frame, persist=True)
    
    if result is not None and result[0].boxes.id is not None:
        track_ids = result[0].boxes.id.int().cpu().tolist()
        boxes = result[0].boxes.xywh.cpu()
        frame_ = result[0].plot()   
        
        return frame_, track_ids, result, boxes
    else:
        return None, None, None, None

In [64]:
def detect_license_plate(detected_area, model,track_ids, result):
    global captured
    global plate
    # Loop untuk setiap hasil deteksi
    for i, track_id in enumerate(track_ids):
        x1, y1, x2, y2, id, score, class_id = result[0].boxes.data[i].numpy()
        if track_id not in captured and int(class_id) != 2:
            capture_frame = detected_area[int(y1):int(y2), int(x1):int(x2)]
            license_detection = model(capture_frame)[0]
            for detection in license_detection.boxes.data.tolist():
                x1_license, y1_license, x2_license, y2_license, score_license, class_id_license = detection
                if class_id_license == 2:
                    license_frame = capture_frame[int(y1_license):int(y2_license), int(x1_license):int(x2_license)]
                    captured.append(track_id)
                    # Ganti fungsionalitas untuk menyimpan gambar, misalnya:
                    cv2.imwrite(f'./license_predict/{plate}.jpg', license_frame)
                    plate += 1

In [65]:
def resized_frame(frame):
    ## MENAMPILKAN LIVE
    # Mendapatkan dimensi asli frame
    original_height, original_width = frame.shape[:2]

    # Hitung faktor skala untuk menyesuaikan frame agar muat dalam satu layar laptop
    screen_height = 720  # Misalnya, resolusi layar laptop adalah 720p
    screen_width = 1280  # Misalnya, resolusi layar laptop adalah 1280p
    scale_factor = min(screen_height / original_height, screen_width / original_width)

    # Resize frame sesuai dengan faktor skala
    resized_frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor)
    
    return resized_frame

In [66]:
ret = True
captured = []
len_file = os.listdir('./license_predict/')
plate = len(len_file)

while ret:
    ret, frame = cap.read()
    
    if ret:
        detected_area, top_left_x, top_left_y, bottom_right_x, bottom_right_y = create_detection_frame(frame)
        
        frame_, track_ids, result, boxes = detect_objects(detected_area, model)
        
        if frame_ is not None:
            detect_license_plate(detected_area, model2, track_ids, result)

            # Pastikan untuk mengganti area yang dicrop dengan frame_
            frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = frame_
            
            cv2.imshow('frame', resized_frame(frame))
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            print("Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.")
            continue  # Lanjutkan ke frame berikutnya jika hasil tidak tersedia
        
cv2.destroyAllWindows()


0: 192x800 2 Mobils, 8.0ms
Speed: 0.5ms preprocess, 8.0ms inference, 1.6ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 2 Mobils, 8.6ms
Speed: 2.0ms preprocess, 8.6ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 2 Mobils, 1 Plat Nomor, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 800)



Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.


0: 192x800 3 Mobils, 1 Plat Nomor, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 6.1ms
Speed: 1.0ms preprocess, 6.1ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 800)

0: 640x800 1 Mobil, 1 Plat Nomor, 14.5ms


Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.


Speed: 3.0ms preprocess, 14.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 800)

0: 192x800 1 Mobil, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 8.6ms
Speed: 2.0ms preprocess, 8.6ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 6.8ms
Speed: 1.0ms preprocess, 6.8ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 6.3ms
Speed: 1.0ms preprocess, 6.3ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 9.5ms
Speed: 1.9ms preprocess, 9.5ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 1 Mobil, 5.6ms
Speed: 1.0ms pr

...

Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.


0: 192x800 3 Mobils, 25.1ms
Speed: 2.9ms preprocess, 25.1ms inference, 4.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 2 Mobils, 25.3ms
Speed: 3.5ms preprocess, 25.3ms inference, 4.2ms postprocess per image at shape (1, 3, 192, 800)



Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.


0: 192x800 2 Mobils, 1 Motor, 1 Plat Nomor, 92.1ms
Speed: 3.0ms preprocess, 92.1ms inference, 4.0ms postprocess per image at shape (1, 3, 192, 800)

0: 192x800 2 Mobils, 52.8ms
Speed: 4.5ms preprocess, 52.8ms inference, 5.0ms postprocess per image at shape (1, 3, 192, 800)


Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.
